In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = True

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.auto import trange
from statistics import mean
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset, random_split, WeightedRandomSampler, Dataset
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel, AutoConfig
import sys
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm.notebook import trange, tqdm
from sklearn.metrics import accuracy_score, classification_report
import os

In [ ]:
'''My edited code'''

# class Embedding(nn.Module):
#     def __init__(self, pretrained):
#         super().__init__()
#         self._pretrained = pretrained
#         self._config = AutoConfig.from_pretrained(self._pretrained)
#         self.bert = AutoModel.from_pretrained(self._pretrained, config=self._config)

#     def _freeze(self):
#         print('Unfreezing the last layer of BERT...')
#         for params in self.bert.parameters():
#             params.requires_grad = False
#         for params in self.bert.encoder.layer[-1].parameters():
#             params.requires_grad = True
#         print('Unfreezed the last layer successfully.')

#     def forward(self, encoded_dict):
#         output = self.bert(**encoded_dict)
#         return output.last_hidden_state[:, 0, :]

# class Classifier(nn.Module):
#     def __init__(self, input_channels=768):
#         super().__init__()
#         self.lin = nn.Linear(input_channels, 3)

#     def forward(self, x):
#         lin = self.lin(x)
#         output = lin
#         return output

# class Model(nn.Module):
#     def __init__(self, pretrained):
#         super().__init__()
#         self.encode = Embedding(pretrained)
#         self.encode._freeze()
#         self.decode = Classifier()

#     def forward(self, encoded_dict):
#         emb = self.encode(encoded_dict)
#         output = self.decode(emb)
#         return output

# class BertDataset(Dataset):
#     def __init__(self, dataframe, pretrained, train=True):
#         self._dataframe = dataframe
#         self._train = train
#         self._tokenizer = AutoTokenizer.from_pretrained(pretrained)

#     def _preprocess(self, text):
#         encoded_dict = self._tokenizer.encode_plus(text=text, add_special_tokens=True, max_length=512, padding='max_length',
#                                                    truncation=True, return_attention_mask=True, return_tensors='pt')
#         return encoded_dict

#     def __getitem__(self, index):
#         encoded_dict = self._preprocess(self._dataframe.iloc[index]['feature'])
#         for el in encoded_dict:
#             encoded_dict[el] = encoded_dict[el].squeeze()
#         encoded_dict['label'] = torch.tensor(self._dataframe.iloc[index]['class'])
#         return encoded_dict

#     def __len__(self):
#         return len(self._dataframe)

# class Main():
#     def __init__(self, device, pretrained, model_file=None):
#         self.loss_fn = nn.CrossEntropyLoss()
#         self.device = device
#         self.pretrained = pretrained
#         self.model = Model(self.pretrained)
#         self.model.to(self.device)
#         if model_file is not None:
#             self.model.load_state_dict(torch.load(model_file))

#     def prepare_dataloader(self, data, train_ratio=0.8, val_ratio=0.1, batchsize=8):
#         dataset = BertDataset(data, self.pretrained)
#         size = len(data)
#         train_size = int(train_ratio * size)
#         val_size = int(val_ratio * size)
#         test_size = size - train_size - val_size
#         train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
#         train_loader = DataLoader(train_dataset, batch_size=batchsize)
#         val_loader = DataLoader(val_dataset, batch_size=batchsize)
#         test_loader = DataLoader(test_dataset, batch_size=batchsize)
#         #torch.save(test_dataset.dataset._dataframe, "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset.pth")

#         return train_loader, val_loader, test_loader

#     def evaluate(self, dataloader):
#         total_loss = 0
#         all_predictions = []
#         all_ground_truth = []

#         for data in dataloader:
#             data = {key: val.to(self.device) for key, val in data.items()}
#             encoded_dict = {key: data[key] for key in data.keys() if key != 'label'}
#             y = data['label'].to(torch.long)
#             y_pred = self.model(encoded_dict)
#             loss = self.loss_fn(y_pred, y)
#             total_loss += loss.item()

#             predictions = torch.argmax(y_pred, dim=1).cpu().numpy()
#             ground_truth = y.cpu().numpy()
#             all_predictions.extend(predictions)
#             all_ground_truth.extend(ground_truth)

#         accuracy = accuracy_score(all_ground_truth, all_predictions)
#         return total_loss / len(dataloader), accuracy

#     def train(self, train_loader, val_loader, test_loader, epochs, model_file, opt=None, lr_scheduler=None):
#         best_dev_loss = sys.maxsize
#         train_losses = []
#         val_losses = []
#         train_accuracies = []
#         val_accuracies = []
#         best_epoch = -99

#         print('Training starting...')
#         for epoch in trange(epochs):
#             self.model.train()
#             loss_val = []
#             avg_loss_epoch = []
#             if epoch == 0:
#                 train_loss, train_accuracy = self.evaluate(train_loader)
#                 val_loss, val_accuracy = self.evaluate(val_loader)
#                 print(
#                     f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}')
#                 train_losses.append(train_loss)
#                 val_losses.append(val_loss)
#                 train_accuracies.append(train_accuracy)
#                 val_accuracies.append(val_accuracy)

#             start = len(val_losses)
#             for data in train_loader:
#                 data = {key: val.to(self.device) for key, val in data.items()}
#                 encoded_dict = {key: data[key] for key in data.keys() if key != 'label'}
#                 y = data['label'].to(torch.long)
#                 y_pred = self.model(encoded_dict)
#                 loss = self.loss_fn(y_pred, y)
#                 loss.backward()
#                 opt.step()
#                 loss_val.append(loss.item())

#             end = len(val_losses)
#             avg_loss_epoch.append(mean(loss_val[start: end + 1]))

#             with torch.no_grad():
#                 self.model.eval()
#                 train_loss, train_accuracy = self.evaluate(train_loader)
#                 val_loss, val_accuracy = self.evaluate(val_loader)

#                 if val_loss < best_dev_loss:
#                     best_dev_loss = val_loss
#                     best_epoch = epoch
#                     torch.save(self.model.state_dict(), model_file)

#                 if lr_scheduler is not None:
#                     lr_scheduler.step(val_loss)

#                 train_losses.append(train_loss)
#                 val_losses.append(val_loss)
#                 train_accuracies.append(train_accuracy)
#                 val_accuracies.append(val_accuracy)

#                 print(
#                     f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}')

#             if epoch - best_epoch > 5:
#                 print('Early stopping....')
#                 break

#         print(
#             f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}, Test Loss: {self.evaluate(test_loader)[0]:.6f}')

#     def predict(self, dataloader):
#         total_loss = 0
#         self.model.eval()
#         self.model.to(self.device)
#         y_class = []
#         ground_truth = []
#         with torch.no_grad():
#             for data in tqdm(dataloader):
#                 data = {key : val.to(self.device) for key, val in data.items()}
#                 encoded_dict = {key : data[key] for key in data.keys() if key != 'label' }
#                 y = data['label'].to(torch.long)
#                 y_pred = self.model(encoded_dict)
#                 #print(y_pred, y_pred.shape)
#                 y_pred_prob = torch.softmax(y_pred, dim = 1)
#                 y_class.extend(torch.argmax(y_pred_prob, dim = 1).detach().cpu().numpy())
#                 ground_truth.extend(y.detach().cpu().numpy())
#                 #print(y_class)
#                 #print(y_pred_prob, y_pred_prob.shape)
#                 loss = self.loss_fn(y_pred, y)
#                 total_loss += loss.item()
#         print('Test loss is %.6f' %(total_loss / len(dataloader)))
#         print('Classification Report')
#         print(classification_report(ground_truth, y_class))

# def run_cls(dataset, device, pretrained, lr=1e-5, weight_decay=0.001,
#             epochs=30, model_file=None, train=True):
#     mainclass = Main(device, pretrained)
#     train_loader, val_loader, test_loader = mainclass.prepare_dataloader(dataset)
#     if train is not True:
#         mainclass.model_file = model_file
#         mainclass.predict(test_loader)
#     else:
#         opt = optim.Adam(mainclass.model.parameters(), lr=lr, weight_decay=weight_decay)
#         lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=4)
#         mainclass.train(train_loader, val_loader, test_loader, epochs, model_file, opt=opt, lr_scheduler=lr_scheduler)


'My edited code'

In [ ]:
'''Neel's code'''

# class Embedding(nn.Module):
#     def __init__(self, pretrained):
#         super().__init__()
#         self._pretrained = pretrained
#         self._config = AutoConfig.from_pretrained(self._pretrained)
#         self.bert = AutoModel.from_pretrained(self._pretrained, config=self._config)

#     def _freeze(self):
#         print('Unfreezing the last layer of BERT...')
#         for params in self.bert.parameters():
#             params.requires_grad = False
#         for params in self.bert.encoder.layer[-1].parameters():
#             params.requires_grad = True
#         print('Unfreezed the last layer successfully.')

#     def forward(self, encoded_dict):
#         output = self.bert(**encoded_dict)
#         return output.last_hidden_state[:, 0, :]

# class Classifier(nn.Module):
#     def __init__(self, input_channels=768):
#         super().__init__()
#         self.lin = nn.Linear(input_channels, 3)

#     def forward(self, x):
#         lin = self.lin(x)
#         output = lin
#         return output

# class Model(nn.Module):
#     def __init__(self, pretrained):
#         super().__init__()
#         self.encode = Embedding(pretrained)
#         self.encode._freeze()
#         self.decode = Classifier()

#     def forward(self, encoded_dict):
#         emb = self.encode(encoded_dict)
#         output = self.decode(emb)
#         return output

# class BertDataset(Dataset):
#     def __init__(self, dataframe, pretrained, train=True):
#         self._dataframe = dataframe
#         self._train = train
#         self._tokenizer = AutoTokenizer.from_pretrained(pretrained)

#     def _preprocess(self, text):
#         encoded_dict = self._tokenizer.encode_plus(text=text, add_special_tokens=True, max_length=512, padding='max_length',
#                                                    truncation=True, return_attention_mask=True, return_tensors='pt')
#         return encoded_dict

#     def __getitem__(self, index):
#         encoded_dict = self._preprocess(self._dataframe.iloc[index]['feature'])
#         for el in encoded_dict:
#             encoded_dict[el] = encoded_dict[el].squeeze()
#         encoded_dict['label'] = torch.tensor(self._dataframe.iloc[index]['class'])
#         return encoded_dict

#     def __len__(self):
#         return len(self._dataframe)

# class TestDataset(Dataset):
#     def __init__(self, dataframe, pretrained):
#         self.dataframe = dataframe
#         self.tokenizer = AutoTokenizer.from_pretrained(pretrained)

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, index):
#         encoded_dict = self._preprocess(self.dataframe.iloc[index]['feature'])
#         for el in encoded_dict:
#             encoded_dict[el] = encoded_dict[el].squeeze()
#         encoded_dict['label'] = torch.tensor(self.dataframe.iloc[index]['class'])
#         return encoded_dict

#     def _preprocess(self, text):
#         encoded_dict = self.tokenizer.encode_plus(text=text, add_special_tokens=True, max_length=512,
#                                                   padding='max_length', truncation=True, return_attention_mask=True,
#                                                   return_tensors='pt')
#         return encoded_dict

# class Main():
#     def __init__(self, device, pretrained, model_file=None):
#         self.loss_fn = nn.CrossEntropyLoss()
#         self.device = device
#         self.pretrained = pretrained
#         self.model = Model(self.pretrained)
#         self.model.to(self.device)
#         if model_file is not None:
#             self.model.load_state_dict(torch.load(model_file))

#     def prepare_dataloader(self, data, train_ratio=0.8, val_ratio=0.1, batchsize=8):
#         dataset = BertDataset(data, self.pretrained)
#         size = len(data)
#         train_size = int(train_ratio * size)
#         val_size = int(val_ratio * size)
#         test_size = size - train_size - val_size
#         train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

#         # Save the test dataset
#         test_dataset.dataset._dataframe.to_pickle("/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset.pth")

#         train_loader = DataLoader(train_dataset, batch_size=batchsize)
#         val_loader = DataLoader(val_dataset, batch_size=batchsize)
#         test_loader = DataLoader(test_dataset, batch_size=batchsize)

#         return train_loader, val_loader, test_loader

#     def evaluate(self, dataloader):
#         total_loss = 0
#         all_predictions = []
#         all_ground_truth = []

#         for data in dataloader:
#             data = {key: val.to(self.device) for key, val in data.items()}
#             encoded_dict = {key: data[key] for key in data.keys() if key != 'label'}
#             y = data['label'].to(torch.long)
#             y_pred = self.model(encoded_dict)
#             loss = self.loss_fn(y_pred, y)
#             total_loss += loss.item()

#             predictions = torch.argmax(y_pred, dim=1).cpu().numpy()
#             ground_truth = y.cpu().numpy()
#             all_predictions.extend(predictions)
#             all_ground_truth.extend(ground_truth)

#         accuracy = accuracy_score(all_ground_truth, all_predictions)
#         return total_loss / len(dataloader), accuracy

#     def train(self, train_loader, val_loader, test_loader, epochs, model_file, opt=None, lr_scheduler=None):
#         best_dev_loss = sys.maxsize
#         train_losses = []
#         val_losses = []
#         train_accuracies = []
#         val_accuracies = []
#         best_epoch = -99

#         print('Training starting...')
#         for epoch in trange(epochs):
#             self.model.train()
#             loss_val = []
#             avg_loss_epoch = []
#             if epoch == 0:
#                 train_loss, train_accuracy = self.evaluate(train_loader)
#                 val_loss, val_accuracy = self.evaluate(val_loader)
#                 print(
#                     f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}')
#                 train_losses.append(train_loss)
#                 val_losses.append(val_loss)
#                 train_accuracies.append(train_accuracy)
#                 val_accuracies.append(val_accuracy)

#             start = len(val_losses)
#             for data in train_loader:
#                 data = {key: val.to(self.device) for key, val in data.items()}
#                 encoded_dict = {key: data[key] for key in data.keys() if key != 'label'}
#                 y = data['label'].to(torch.long)
#                 y_pred = self.model(encoded_dict)
#                 loss = self.loss_fn(y_pred, y)
#                 loss.backward()
#                 opt.step()
#                 loss_val.append(loss.item())

#             end = len(val_losses)
#             avg_loss_epoch.append(mean(loss_val[start: end + 1]))

#             with torch.no_grad():
#                 self.model.eval()
#                 train_loss, train_accuracy = self.evaluate(train_loader)
#                 val_loss, val_accuracy = self.evaluate(val_loader)

#                 if val_loss < best_dev_loss:
#                     best_dev_loss = val_loss
#                     best_epoch = epoch
#                     torch.save(self.model.state_dict(), model_file)

#                 if lr_scheduler is not None:
#                     lr_scheduler.step(val_loss)

#                 train_losses.append(train_loss)
#                 val_losses.append(val_loss)
#                 train_accuracies.append(train_accuracy)
#                 val_accuracies.append(val_accuracy)

#                 print(
#                     f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}')

#             if epoch - best_epoch > 5:
#                 print('Early stopping....')
#                 break

#         print(
#             f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}, Test Loss: {self.evaluate(test_loader)[0]:.6f}')

#     def predict(self, dataloader):
#         total_loss = 0
#         self.model.eval()
#         self.model.to(self.device)
#         y_class = []
#         ground_truth = []
#         with torch.no_grad():
#             for data in tqdm(dataloader):
#                 data = {key: val.to(self.device) for key, val in data.items()}
#                 encoded_dict = {key: data[key] for key in data.keys() if key != 'label'}
#                 y = data['label'].to(torch.long)
#                 y_pred = self.model(encoded_dict)
#                 y_pred_prob = torch.softmax(y_pred, dim=1)
#                 y_class.extend(torch.argmax(y_pred_prob, dim=1).detach().cpu().numpy())
#                 ground_truth.extend(y.detach().cpu().numpy())
#                 loss = self.loss_fn(y_pred, y)
#                 total_loss += loss.item()
#         print('Test loss is %.6f' % (total_loss / len(dataloader)))
#         print('Classification Report')
#         print(classification_report(ground_truth, y_class))

# def run_cls(dataset, device, pretrained, lr=1e-5, weight_decay=0.001,
#             epochs=30, model_file=None, train=True):
#     mainclass = Main(device, pretrained)
#     train_loader, val_loader, test_loader = mainclass.prepare_dataloader(dataset)

#     if not train:
#         test_dataset = pd.read_pickle("/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset.pth")
#         custom_test_dataset = TestDataset(test_dataset, pretrained)
#         test_loader = DataLoader(custom_test_dataset, batch_size=8, shuffle=False)

#         mainclass.model_file = model_file
#         mainclass.predict(test_loader)
#     else:
#         opt = optim.Adam(mainclass.model.parameters(), lr=lr, weight_decay=weight_decay)
#         lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=4)
#         mainclass.train(train_loader, val_loader, test_loader, epochs, model_file, opt=opt, lr_scheduler=lr_scheduler)

"Neel's code"

In [ ]:
'''Working Code'''

class Embedding(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self._pretrained = pretrained
        self._config = AutoConfig.from_pretrained(self._pretrained)
        self.bert = AutoModel.from_pretrained(self._pretrained, config=self._config)

    def _freeze(self):
        print('Unfreezing the last layer of BERT...')
        for params in self.bert.parameters():
            params.requires_grad = False

        if 'distil' in self._pretrained.lower():
            print('Distillation...')

        else:
            for params in self.bert.encoder.layer[-1].parameters():
                params.requires_grad = True
        print('Unfreezed the last layer successfully.')


    def forward(self, encoded_dict):
        output = self.bert(**encoded_dict)
        return output.last_hidden_state[:, 0, :]

class Classifier(nn.Module):
    def __init__(self, input_channels=768):
        super().__init__()
        self.lin = nn.Linear(input_channels, 3)

    def forward(self, x):
        lin = self.lin(x)
        output = lin
        return output

class Model(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.encode = Embedding(pretrained)
        self.encode._freeze()
        self.decode = Classifier()

    def forward(self, encoded_dict):
        emb = self.encode(encoded_dict)
        output = self.decode(emb)
        return output

class BertDataset(Dataset):
    def __init__(self, dataframe, pretrained, train=True):
        self._dataframe = dataframe
        self._train = train
        self._tokenizer = AutoTokenizer.from_pretrained(pretrained)

    def _preprocess(self, text):
        encoded_dict = self._tokenizer.encode_plus(text=text, add_special_tokens=True, max_length=512, padding='max_length',
                                                   truncation=True, return_attention_mask=True, return_tensors='pt')
        return encoded_dict

    def __getitem__(self, index):
        encoded_dict = self._preprocess(self._dataframe.iloc[index]['feature'])
        for el in encoded_dict:
            encoded_dict[el] = encoded_dict[el].squeeze()
        encoded_dict['label'] = torch.tensor(self._dataframe.iloc[index]['class'])
        return encoded_dict

    def __len__(self):
        return len(self._dataframe)

class TestDataset(Dataset):
    def __init__(self, dataframe, pretrained):
        self.dataframe = dataframe
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        encoded_dict = self._preprocess(self.dataframe.iloc[index]['feature'])
        for el in encoded_dict:
            encoded_dict[el] = encoded_dict[el].squeeze()
        encoded_dict['label'] = torch.tensor(self.dataframe.iloc[index]['class'])
        return encoded_dict

    def _preprocess(self, text):
        encoded_dict = self.tokenizer.encode_plus(text=text, add_special_tokens=True, max_length=512,
                                                  padding='max_length', truncation=True, return_attention_mask=True,
                                                  return_tensors='pt')
        return encoded_dict

class Main():
    def __init__(self, device, pretrained, model_file=None):
        self.loss_fn = nn.CrossEntropyLoss()
        self.device = device
        self.pretrained = pretrained
        self.model = Model(self.pretrained)
        self.model.to(self.device)
        if model_file is not None:
            if os.path.exists(model_file):
                self.model.load_state_dict(torch.load(model_file, map_location = self.device))

    def prepare_dataloader(self, data, train_ratio=0.8, val_ratio=0.1, batchsize=8, test_dataset_path=None):
        dataset = BertDataset(data, self.pretrained)
        size = len(data)
        train_size = int(train_ratio * size)
        val_size = int(val_ratio * size)
        test_size = size - train_size - val_size
        train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

        if test_dataset_path is not None:
            test_dataset.dataset._dataframe.to_pickle(test_dataset_path)

        train_loader = DataLoader(train_dataset, batch_size=batchsize)
        val_loader = DataLoader(val_dataset, batch_size=batchsize)
        test_loader = DataLoader(test_dataset, batch_size=batchsize)

        return train_loader, val_loader, test_loader

    def evaluate(self, dataloader):
        total_loss = 0
        all_predictions = []
        all_ground_truth = []

        for data in dataloader:
            data = {key: val.to(self.device) for key, val in data.items()}
            encoded_dict = {key: data[key] for key in data.keys() if key != 'label'}
            y = data['label'].to(torch.long)
            y_pred = self.model(encoded_dict)
            loss = self.loss_fn(y_pred, y)
            total_loss += loss.item()

            predictions = torch.argmax(y_pred, dim=1).cpu().numpy()
            ground_truth = y.cpu().numpy()
            all_predictions.extend(predictions)
            all_ground_truth.extend(ground_truth)

        accuracy = accuracy_score(all_ground_truth, all_predictions)
        return total_loss / len(dataloader), accuracy

    def train(self, train_loader, val_loader, test_loader, epochs, model_file, opt=None, lr_scheduler=None):
        best_dev_loss = sys.maxsize
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []
        best_epoch = -99

        print('Training starting...')
        for epoch in trange(epochs):
            self.model.train()
            loss_val = []
            avg_loss_epoch = []
            if epoch == 0:
                train_loss, train_accuracy = self.evaluate(train_loader)
                val_loss, val_accuracy = self.evaluate(val_loader)
                print(
                    f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}')
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                train_accuracies.append(train_accuracy)
                val_accuracies.append(val_accuracy)

            start = len(val_losses)
            for data in train_loader:
                data = {key: val.to(self.device) for key, val in data.items()}
                encoded_dict = {key: data[key] for key in data.keys() if key != 'label'}
                y = data['label'].to(torch.long)
                y_pred = self.model(encoded_dict)
                loss = self.loss_fn(y_pred, y)
                loss.backward()
                opt.step()
                loss_val.append(loss.item())

            end = len(val_losses)
            avg_loss_epoch.append(mean(loss_val[start: end + 1]))

            with torch.no_grad():
                self.model.eval()
                train_loss, train_accuracy = self.evaluate(train_loader)
                val_loss, val_accuracy = self.evaluate(val_loader)

                if val_loss < best_dev_loss:
                    best_dev_loss = val_loss
                    best_epoch = epoch
                    torch.save(self.model.state_dict(), model_file)

                if lr_scheduler is not None:
                    lr_scheduler.step(val_loss)

                train_losses.append(train_loss)
                val_losses.append(val_loss)
                train_accuracies.append(train_accuracy)
                val_accuracies.append(val_accuracy)

                print(
                    f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}')

            if epoch - best_epoch > 5:
                print('Early stopping....')
                break

        print(
            f'Epoch {epoch} / {epochs}: Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}, Test Loss: {self.evaluate(test_loader)[0]:.6f}')

    def predict(self, dataloader):
        total_loss = 0
        self.model.eval()
        self.model.to(self.device)
        y_class = []
        ground_truth = []
        with torch.no_grad():
            for data in tqdm(dataloader):
                data = {key: val.to(self.device) for key, val in data.items()}
                encoded_dict = {key: data[key] for key in data.keys() if key != 'label'}
                y = data['label'].to(torch.long)
                y_pred = self.model(encoded_dict)
                y_pred_prob = torch.softmax(y_pred, dim=1)
                y_class.extend(torch.argmax(y_pred_prob, dim=1).detach().cpu().numpy())
                ground_truth.extend(y.detach().cpu().numpy())
                loss = self.loss_fn(y_pred, y)
                total_loss += loss.item()
        print('Test loss is %.6f' % (total_loss / len(dataloader)))
        print('Classification Report')
        print(classification_report(ground_truth, y_class))

def run_cls(dataset, device, pretrained, lr=1e-5, weight_decay=0.001,
            epochs=30, model_file=None, train=True, train_file_path=None, test_file_path=None):

    mainclass = Main(device, pretrained, model_file=model_file)

    if train:
        train_loader, val_loader, test_loader = mainclass.prepare_dataloader(dataset, test_dataset_path=test_file_path)

        opt = optim.Adam(mainclass.model.parameters(), lr=lr, weight_decay=weight_decay)
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=4)
        mainclass.train(train_loader, val_loader, test_loader, epochs, model_file, opt=opt, lr_scheduler=lr_scheduler)
    else:
        test_dataset = pd.read_pickle(test_file_path)
        custom_test_dataset = TestDataset(test_dataset, pretrained)
        test_loader = DataLoader(custom_test_dataset, batch_size=8, shuffle=False)

        mainclass.model_file = model_file
        mainclass.predict(test_loader)


# **English**

## **English Imbalanced OG Dataset**

In [ ]:
filename = '/content/drive/MyDrive/LREC-Coling/Final_Data/English.csv'
data = pd.read_csv(filename)
data

,URL,news_title,news_content,impact_level,impact_length,Text_korean,Text_japanese,Text_french
0,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,ESG-focused financial technology company Arabe...,low,2 to 5 years,ESG 중심의 금융 기술 회사 인 Arabesque AI는 오늘 회사의 새로운 최고...,ESGに焦点を当てた金融テクノロジー企業のアラブスクAIは本日、カロライナミニオパルエロ博士...,La société de technologies financières axée su...
1,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,The company also announced the appointment of ...,low,2 to 5 years,이 회사는 또한 Tim Wong을 새로운 최고 제품 책임자로 임명했다고 발표했습니다...,同社はまた、ティム・ウォンを新しい最高製品責任者に任命することを発表しました。 Wongは、...,La société a également annoncé la nomination d...
2,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,Wong said: \n“Personalised portfolios demand ...,medium,2 to 5 years,Wong은 다음과 같이 말했습니다 : \ n 개인화 된 포트폴리오는 일반적인 자산 ...,Wong氏は次のように述べています。「パーソナライズされたポートフォリオには、典型的な資産マ...,Wong a déclaré: \ n «Des portefeuilles personn...
3,https://www.esgtoday.com/ukraine-war-inflation...,"Ukraine War, Inflation Reduction Act Driving F...",One of the key themes of the report is the imp...,high,More than 5 years,이 보고서의 주요 주제 중 하나는 러시아-우크라이나 전쟁으로 인한 글로벌 에너지 공...,報告書の重要なテーマの1つは、世界のエネルギー供給の混乱とロシア - ウクレーン戦争によって...,L'un des thèmes clés du rapport est l'impact s...
4,https://www.esgtoday.com/eu-regulators-welcome...,"EU Regulators Welcome, Critique New European S...",Europe’s three primary financial regulatory ag...,medium,Less than 2 years,유럽의 3 가지 주요 금융 규제 기관인 ESAS (European Superviso...,ヨーロッパの3つの主要な金融規制機関である欧州監督当局（ESA）は、それぞれ、欧州の持続可能...,Les trois principales agences de réglementatio...
...,...,...,...,...,...,...,...,...
540,https://www.esgtoday.com/methane-emissions-det...,Methane Emissions Detection Platform Kuva Rais...,"Stefan Bokaemper, CEO of Kuva Systems, said: “...",low,2 to 5 years,Kuva Systems의 CEO 인 Stefan Bokaemper는 다음과 같이 말...,Kuva SystemsのCEOであるStefan Bokaemper氏は次のように述べてい...,"Stefan Bokaemper, PDG de Kuva Systems, a décla..."
541,https://www.esgtoday.com/eaton-appoints-harold...,Eaton Appoints Harold Jones as Chief Sustainab...,Eaton Appoints Harold Jones as Chief Sustainab...,low,2 to 5 years,이튼은 해롤드 존스를 최고 지속 가능성 책임자로 임명합니다,イートンは、ハロルド・ジョーンズを最高の持続可能性責任者に任命します,Eaton nomme Harold Jones en tant que chef de l...
542,https://www.esgtoday.com/ssga-outlines-2021-st...,"SSGA Outlines 2021 Stewardship Priorities, Wil...","In his letter, Taraporevala wrote: “As a signa...",medium,Less than 2 years,Taraporevala는 그의 편지에서 다음과 같이 썼습니다. 2021 년에 우리는...,彼の手紙の中で、Taraporevalaは次のように書いています。 2021年には、気候変動...,"Dans sa lettre, Taraporevala a écrit: «En tant..."
543,https://www.esgtoday.com/survey-investors-shif...,Survey: Investors Shifting to Offense on Clima...,O’Brien said: “Investors globally are increasi...,low,Less than 2 years,오브라이언은 다음과 같이 말했습니다 :“전 세계 투자자들은 직장에 다양성과 포용이 ...,オブライエンは次のように述べています。「世界的には、職場への多様性と包含の欠如がもたらすビジ...,"O’Brien a déclaré: ""Les investisseurs dans le ..."


In [ ]:
data.rename(columns={'news_content': 'feature', 'impact_level': 'class'}, inplace=True)
display(data['class'].value_counts())
dataset = data[['feature', 'class']]
key = {}
key['high'] = 0
key['low'] = 1
key['medium'] = 2
dataset['class'] = dataset['class'].apply(lambda x : key[x])
dataset

medium    243
high      196
low       106
Name: class, dtype: int64

<ipython-input-34-f2b88783b9d2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['class'] = dataset['class'].apply(lambda x : key[x])


,feature,class
0,ESG-focused financial technology company Arabe...,1
1,The company also announced the appointment of ...,1
2,Wong said: \n“Personalised portfolios demand ...,2
3,One of the key themes of the report is the imp...,0
4,Europe’s three primary financial regulatory ag...,2
...,...,...
540,"Stefan Bokaemper, CEO of Kuva Systems, said: “...",1
541,Eaton Appoints Harold Jones as Chief Sustainab...,1
542,"In his letter, Taraporevala wrote: “As a signa...",2
543,O’Brien said: “Investors globally are increasi...,1


### ***bert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/english_imbalanced_bert_base_uncased_finetuned.pt'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/test_dataset_english_imbalanced_bert_base_uncased_finetuned.pth"

pretrained = 'bert-base-uncased'

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/69 [00:00<?, ?it/s]

Test loss is 0.737914
Classification Report
              precision    recall  f1-score   support

           0       0.69      0.83      0.75       196
           1       0.59      0.71      0.64       106
           2       0.77      0.58      0.66       243

    accuracy                           0.69       545
   macro avg       0.68      0.70      0.68       545
weighted avg       0.71      0.69      0.69       545



### ***ProsusAI/finbert***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/english_imbalanced_ProsusAI_finbert_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/test_dataset_english_imbalanced_ProsusAI_finbert_finetuned.pth"

pretrained = 'ProsusAI/finbert'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/69 [00:00<?, ?it/s]

Test loss is 0.668163
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.59      0.69       196
           1       0.97      0.36      0.52       106
           2       0.61      0.92      0.73       243

    accuracy                           0.69       545
   macro avg       0.81      0.62      0.65       545
weighted avg       0.76      0.69      0.68       545



### ***dlxyuan/distilbert-base-multilingual-cased-sentiments-student***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/test_dataset_english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pth"

pretrained = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Distillation...
Unfreezed the last layer successfully.


  0%|          | 0/69 [00:00<?, ?it/s]

Test loss is 1.017083
Classification Report
              precision    recall  f1-score   support

           0       0.55      0.21      0.30       196
           1       0.00      0.00      0.00       106
           2       0.46      0.89      0.61       243

    accuracy                           0.47       545
   macro avg       0.34      0.37      0.30       545
weighted avg       0.40      0.47      0.38       545



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### ***distilbert/distilbert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/english_imbalanced_distilbert-base-uncased_finetuned.pt'
pretrained = 'distilbert/distilbert-base-uncased'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/test_dataset_english_imbalanced_distilbert-base-uncased_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Distillation...
Unfreezed the last layer successfully.


  0%|          | 0/69 [00:00<?, ?it/s]

Test loss is 0.868830
Classification Report
              precision    recall  f1-score   support

           0       0.65      0.61      0.63       196
           1       0.80      0.26      0.40       106
           2       0.56      0.74      0.64       243

    accuracy                           0.60       545
   macro avg       0.67      0.54      0.55       545
weighted avg       0.64      0.60      0.59       545



### ***cross-encoder/nli-distilroberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/english_imbalanced_nli-distilroberta-base_finetuned.pt'
pretrained = 'cross-encoder/nli-distilroberta-base'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/test_dataset_english_imbalanced_nli-distilroberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

Some weights of RobertaModel were not initialized from the model checkpoint at cross-encoder/nli-distilroberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Unfreezing the last layer of BERT...
Distillation...
Unfreezed the last layer successfully.


  0%|          | 0/69 [00:00<?, ?it/s]

Test loss is 1.057363
Classification Report
              precision    recall  f1-score   support

           0       0.35      0.50      0.41       196
           1       0.00      0.00      0.00       106
           2       0.46      0.50      0.48       243

    accuracy                           0.40       545
   macro avg       0.27      0.33      0.30       545
weighted avg       0.33      0.40      0.36       545



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### ***mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pt'
pretrained = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/test_dataset_english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

Some weights of RobertaModel were not initialized from the model checkpoint at mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Unfreezing the last layer of BERT...
Distillation...
Unfreezed the last layer successfully.


  0%|          | 0/69 [00:00<?, ?it/s]

Test loss is 0.920260
Classification Report
              precision    recall  f1-score   support

           0       0.71      0.47      0.57       196
           1       0.83      0.27      0.41       106
           2       0.55      0.86      0.67       243

    accuracy                           0.61       545
   macro avg       0.70      0.54      0.55       545
weighted avg       0.66      0.61      0.58       545



### ***FacebookAI/roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/english_imbalanced_FacebookAI_roberta-base_finetuned.pt'
pretrained = 'FacebookAI/roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/test_dataset_english_imbalanced_roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/69 [00:00<?, ?it/s]

Test loss is 0.562596
Classification Report
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       196
           1       0.95      0.50      0.65       106
           2       0.74      0.88      0.81       243

    accuracy                           0.80       545
   macro avg       0.85      0.75      0.77       545
weighted avg       0.82      0.80      0.79       545



### ***FacebookAI/xlm-roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/english_imbalanced_FacebookAI_xlm-roberta-base_finetuned.pt'
pretrained = 'FacebookAI/xlm-roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishOG/test_dataset_english_imbalanced_xlm-roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/69 [00:00<?, ?it/s]

Test loss is 0.759824
Classification Report
              precision    recall  f1-score   support

           0       0.69      0.67      0.68       196
           1       0.61      0.54      0.57       106
           2       0.59      0.63      0.61       243

    accuracy                           0.63       545
   macro avg       0.63      0.61      0.62       545
weighted avg       0.63      0.63      0.63       545



## **English Imbalanced Paraphrased (10k)**
### *Model - {*humarin/chatgpt_paraphraser_on_T5_base*}*

In [ ]:
filename = '/content/drive/MyDrive/LREC-Coling/Paraphrased_Data/English_paraphrased.xlsx'
data = pd.read_excel(filename)
data.drop_duplicates(inplace=True)
data.reset_index(inplace = True, drop=True)
data

,URL,news_title,news_content,impact_level,impact_length
0,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,"Arabesque AI, a financial technology company d...",low,2 to 5 years
1,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,Dr. Carolina Minio Paluello has been appointed...,low,2 to 5 years
2,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,The appointment of Dr. Carolina Minio Paluello...,low,2 to 5 years
3,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,"Arabesque AI, a financial technology firm comm...",low,2 to 5 years
4,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,Dr. Carolina Minio Paluello was appointed as t...,low,2 to 5 years
...,...,...,...,...,...
2720,https://www.esgtoday.com/chicago-to-transition...,"Chicago to Transition Buildings, Airports & Op...","According to Jim McHugh, Constellation is deli...",medium,2 to 5 years
2721,https://www.esgtoday.com/chicago-to-transition...,"Chicago to Transition Buildings, Airports & Op...","Constellation's Chief Commercial Officer, Jim ...",medium,2 to 5 years
2722,https://www.esgtoday.com/chicago-to-transition...,"Chicago to Transition Buildings, Airports & Op...",The City of Chicago will benefit from Constell...,medium,2 to 5 years
2723,https://www.esgtoday.com/chicago-to-transition...,"Chicago to Transition Buildings, Airports & Op...","Jim McHugh, CEO of Constellation, stated that ...",medium,2 to 5 years


In [ ]:
data.rename(columns={'news_content': 'feature', 'impact_level': 'class'}, inplace=True)
display(data['class'].value_counts())
dataset = data[['feature', 'class']]
key = {}
key['high'] = 0
key['low'] = 1
key['medium'] = 2
dataset['class'] = dataset['class'].apply(lambda x : key[x])
dataset

medium    1215
high       980
low        530
Name: class, dtype: int64

<ipython-input-7-f2b88783b9d2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['class'] = dataset['class'].apply(lambda x : key[x])


,feature,class
0,"Arabesque AI, a financial technology company d...",1
1,Dr. Carolina Minio Paluello has been appointed...,1
2,The appointment of Dr. Carolina Minio Paluello...,1
3,"Arabesque AI, a financial technology firm comm...",1
4,Dr. Carolina Minio Paluello was appointed as t...,1
...,...,...
2720,"According to Jim McHugh, Constellation is deli...",2
2721,"Constellation's Chief Commercial Officer, Jim ...",2
2722,The City of Chicago will benefit from Constell...,2
2723,"Jim McHugh, CEO of Constellation, stated that ...",2


### ***bert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/english_paraphrased_imbalanced_bert_base_uncased_finetuned.pt'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/test_dataset_english_paraphrased_imbalanced_bert_base_uncased_finetuned.pth"

pretrained = 'bert-base-uncased'

run_cls(dataset, device, pretrained, model_file=model_file, train=True, test_file_path=test_file_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/1362 [00:00<?, ?it/s]

KeyboardInterrupt: 

### ***ProsusAI/finbert***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/english_paraphrased_imbalanced_ProsusAI_finbert_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/test_dataset_english_paraphrased_imbalanced_ProsusAI_finbert_finetuned.pth"

pretrained = 'ProsusAI/finbert'
run_cls(dataset, device, pretrained, model_file=model_file, train=False, test_file_path=test_file_path)

### ***dlxyuan/distilbert-base-multilingual-cased-sentiments-student***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/english_paraphrased_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/test_dataset_english_paraphrased_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pth"

pretrained = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***distilbert/distilbert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/english_paraphrased_imbalanced_distilbert-base-uncased_finetuned.pt'
pretrained = 'distilbert/distilbert-base-uncased'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/test_dataset_english_paraphrased_imbalanced_distilbert-base-uncased_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***cross-encoder/nli-distilroberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/english_paraphrased_imbalanced_nli-distilroberta-base_finetuned.pt'
pretrained = 'cross-encoder/nli-distilroberta-base'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/test_dataset_english_paraphrased_imbalanced_nli-distilroberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/englishP/exp1/english_paraphrased_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pt'
pretrained = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/test_dataset_english_paraphrased_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/english_paraphrased_imbalanced_FacebookAI_roberta-base_finetuned.pt'
pretrained = 'FacebookAI/roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/test_dataset_english_paraphrased_imbalanced_roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/xlm-roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/english_paraphrased_imbalanced_FacebookAI_xlm-roberta-base_finetuned.pt'
pretrained = 'FacebookAI/xlm-roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/exp1/englishP/test_dataset_english_paraphrased_imbalanced_xlm-roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

# **Korean**

## **Korean Imbalanced OG Dataset (English)**

In [ ]:
filename = '/content/drive/MyDrive/LREC-Coling/Final_Data/Korean.csv'
data = pd.read_csv(filename)
data.drop_duplicates(inplace=True)
data.reset_index(inplace = True, drop=True)
data.head(5)

,url,category,title,content,impact_type,impact_duration,category.1,title.1,Text_english,Text_japanese,Text_french
0,https://www.esgeconomy.com/news/articleView.ht...,지속가능경제,액화수소 시대 열린다...연 4만톤 설비 연내 가동,"연내에 두산애너빌리티와 SK E&S, 효성중공업의 액화수소 플랜트가 준공돼 연간 최...",opportunity,2 to 5 years,Sustainable economy,LCD hydrogen era opens ... Operation within th...,"Within the year, the liquefaction plant of Doo...",1年以内に、Doosan Anna Village、SK E＆S、およびHyosung重工業...,"Au cours de l'année, l'usine de liquéfaction d..."
1,https://www.esgeconomy.com/news/articleView.ht...,기업·거버넌스,"SK텔레콤, 자사주 소각 통해 지배구조 개편 한걸음",올해 하반기까지 사업재편을 완성하겠다는 SK텔레콤이 주주가치 제고를 위한 자사주 소...,opportunity,less than 2 years,Corporate and governance,SK Telecom's reorganization of the governance ...,"SK Telecom, which will complete its business r...",SK Telecomは、今年の後半までに事業の再編成を完了し、株主価値を高めるために財務省株...,"SK Telecom, qui terminera sa réorganisation co..."
2,https://www.esgeconomy.com/news/articleView.ht...,기업·거버넌스,500대 기업 중 공급망 ESG 관리 들어간 곳 27%뿐,유럽연합(EU)의 공급망실사법 도입과 공급망 온실가스 배출량 공시의무화 움직임 등을...,risk,less than 2 years,Corporate and governance,Only 27%of the 500 companies entered the suppl...,Considering the introduction of the European U...,欧州連合のサプライチェーン法の導入と、サプライチェーン温室効果ガスの排出量の義務的開示を考慮...,Compte tenu de l'introduction de la loi d'écon...
3,https://www.esgeconomy.com/news/articleView.ht...,환경·사회,국제 탄소배출권 가격 연일 사상최고치...유럽시장 70유로선 돌파,유럽연합(EU) 시장을 필두로 국제 탄소배출권 가격이 연일 사상최고치 행진을 이어가...,risk,2 to 5 years,Environment and society,International carbon emission prices are all r...,"Leading the European Union (EU) market, intern...",欧州連合（EU）市場を率いる国際炭素排出価格は、毎日行進し続けています。ヨーロッパの大気量の...,À la direction du marché de l'Union européenne...
4,https://www.esgeconomy.com/news/articleView.ht...,환경·사회,美 서부의 생명수 '콜로라도강'을 지켜라,그랜드캐니언을 관통하면서 미 서부의 생명수 역할을 하는 콜로라도강에 비상이 걸렸다....,risk,2 to 5 years,Environment and society,Protect the Western life of Colorado River,"As he penetrated the Grand Canyon, he had an e...",彼がグランドキャニオンに侵入したとき、彼はコロラド川で緊急事態を抱いていました。これは、米国...,"Alors qu'il pénétrait le Grand Canyon, il avai..."


In [ ]:
data.rename(columns={'Text_english': 'feature', 'impact_type': 'class'}, inplace=True)
display(data['class'].value_counts())
dataset = data[['feature', 'class']]
key = {}
key['opportunity'] = 0
key['risk'] = 1
key['cannot distinguish'] = 2
dataset['class'] = dataset['class'].apply(lambda x : key[x])
dataset

opportunity           452
risk                  220
cannot distinguish    108
Name: class, dtype: int64

<ipython-input-10-e5b5a984f8ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['class'] = dataset['class'].apply(lambda x : key[x])


,feature,class
0,"Within the year, the liquefaction plant of Doo...",0
1,"SK Telecom, which will complete its business r...",0
2,Considering the introduction of the European U...,1
3,"Leading the European Union (EU) market, intern...",1
4,"As he penetrated the Grand Canyon, he had an e...",1
...,...,...
775,"There will be a plan to cultivate 80,000 peopl...",0
776,"Google Pixel Phone, which has a 3% share in th...",2
777,"This year, the global food industry's keywords...",0
778,In order to achieve the goal set by the Paris ...,1


### ***bert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_bert_base_uncased_finetuned.pt'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_imbalanced_bert_base_uncased_finetuned.pth"

pretrained = 'bert-base-uncased'

run_cls(dataset, device, pretrained, model_file=model_file, train=False, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/98 [00:00<?, ?it/s]

Test loss is 0.731808
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.75      0.77       452
           1       0.51      0.83      0.63       220
           2       0.00      0.00      0.00       108

    accuracy                           0.67       780
   macro avg       0.44      0.53      0.47       780
weighted avg       0.61      0.67      0.63       780



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### ***ProsusAI/finbert***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_ProsusAI_finbert_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_dataset_imbalanced_ProsusAI_finbert_finetuned.pth"

pretrained = 'ProsusAI/finbert'
run_cls(dataset, device, pretrained, model_file=model_file, train= False, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/98 [00:00<?, ?it/s]

Test loss is 0.449474
Classification Report
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       452
           1       0.73      0.85      0.78       220
           2       0.78      0.47      0.59       108

    accuracy                           0.82       780
   macro avg       0.80      0.74      0.75       780
weighted avg       0.82      0.82      0.82       780



### ***dlxyuan/distilbert-base-multilingual-cased-sentiments-student***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pth"

pretrained = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***distilbert/distilbert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_distilbert-base-uncased_finetuned.pt'
pretrained = 'distilbert/distilbert-base-uncased'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_imbalanced_distilbert-base-uncased_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***cross-encoder/nli-distilroberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_nli-distilroberta-base_finetuned.pt'
pretrained = 'cross-encoder/nli-distilroberta-base'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_imbalanced_nli-distilroberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pt'
pretrained = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_FacebookAI_roberta-base_finetuned.pt'
pretrained = 'FacebookAI/roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_imbalanced_roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/xlm-roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_FacebookAI_xlm-roberta-base_finetuned.pt'
pretrained = 'FacebookAI/xlm-roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_imbalanced_xlm-roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

## **Korean Imbalanced Parapharsed (English)**

In [ ]:
filename = '/content/drive/MyDrive/LREC-Coling/Paraphrased_Data/Korean_paraphrased.csv'
data = pd.read_csv(filename)
data.drop_duplicates(inplace=True)
data.reset_index(inplace = True, drop=True)
data.head(5)

,url,category,title,content,impact_type,impact_duration,category.1,title.1,Text_english,Text_japanese,Text_french
0,https://www.esgeconomy.com/news/articleView.ht...,지속가능경제,액화수소 시대 열린다...연 4만톤 설비 연내 가동,"연내에 두산애너빌리티와 SK E&S, 효성중공업의 액화수소 플랜트가 준공돼 연간 최...",opportunity,2 to 5 years,Sustainable economy,LCD hydrogen era opens ... Operation within th...,"SK E & S, Hyosung Heavy Industries, and Dooska...",1年以内に、Doosan Anna Village、SK E＆S、およびHyosung重工業...,"Au cours de l'année, l'usine de liquéfaction d..."
1,https://www.esgeconomy.com/news/articleView.ht...,지속가능경제,액화수소 시대 열린다...연 4만톤 설비 연내 가동,"연내에 두산애너빌리티와 SK E&S, 효성중공업의 액화수소 플랜트가 준공돼 연간 최...",opportunity,2 to 5 years,Sustainable economy,LCD hydrogen era opens ... Operation within th...,"To produce up to 40,000 tons of liquefaction b...",1年以内に、Doosan Anna Village、SK E＆S、およびHyosung重工業...,"Au cours de l'année, l'usine de liquéfaction d..."
2,https://www.esgeconomy.com/news/articleView.ht...,지속가능경제,액화수소 시대 열린다...연 4만톤 설비 연내 가동,"연내에 두산애너빌리티와 SK E&S, 효성중공업의 액화수소 플랜트가 준공돼 연간 최...",opportunity,2 to 5 years,Sustainable economy,LCD hydrogen era opens ... Operation within th...,"The liquefaction plant of Doosan Anna Village,...",1年以内に、Doosan Anna Village、SK E＆S、およびHyosung重工業...,"Au cours de l'année, l'usine de liquéfaction d..."
3,https://www.esgeconomy.com/news/articleView.ht...,지속가능경제,액화수소 시대 열린다...연 4만톤 설비 연내 가동,"연내에 두산애너빌리티와 SK E&S, 효성중공업의 액화수소 플랜트가 준공돼 연간 최...",opportunity,2 to 5 years,Sustainable economy,LCD hydrogen era opens ... Operation within th...,"The completion of Doosan Anna Village, SK E & ...",1年以内に、Doosan Anna Village、SK E＆S、およびHyosung重工業...,"Au cours de l'année, l'usine de liquéfaction d..."
4,https://www.esgeconomy.com/news/articleView.ht...,지속가능경제,액화수소 시대 열린다...연 4만톤 설비 연내 가동,"연내에 두산애너빌리티와 SK E&S, 효성중공업의 액화수소 플랜트가 준공돼 연간 최...",opportunity,2 to 5 years,Sustainable economy,LCD hydrogen era opens ... Operation within th...,"Doosan Anna Village, SK E W&S and Hyosumung He...",1年以内に、Doosan Anna Village、SK E＆S、およびHyosung重工業...,"Au cours de l'année, l'usine de liquéfaction d..."


In [ ]:
data.rename(columns={'Text_english': 'feature', 'impact_type': 'class'}, inplace=True)
display(data['class'].value_counts())
dataset = data[['feature', 'class']]
key = {}
key['opportunity'] = 0
key['risk'] = 1
key['cannot distinguish'] = 2
dataset['class'] = dataset['class'].apply(lambda x : key[x])
dataset

opportunity           2260
risk                  1100
cannot distinguish     540
Name: class, dtype: int64

<ipython-input-8-e5b5a984f8ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['class'] = dataset['class'].apply(lambda x : key[x])


,feature,class
0,"SK E & S, Hyosung Heavy Industries, and Dooska...",0
1,"To produce up to 40,000 tons of liquefaction b...",0
2,"The liquefaction plant of Doosan Anna Village,...",0
3,"The completion of Doosan Anna Village, SK E & ...",0
4,"Doosan Anna Village, SK E W&S and Hyosumung He...",0
...,...,...
3895,Many countries around the world are attempting...,0
3896,"To address climate change, many countries arou...",0
3897,Despite efforts to transform energy to address...,0
3898,"In response to climate change, many countries ...",0


### ***bert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_paraphrased_imbalanced_bert_base_uncased_finetuned.pt'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_paraphrased_imbalanced_bert_base_uncased_finetuned.pth"

pretrained = 'bert-base-uncased'

run_cls(dataset, device, pretrained, model_file=model_file, train=False, test_file_path=test_file_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/488 [00:00<?, ?it/s]

Test loss is 0.136832
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      2260
           1       0.95      0.91      0.93      1100
           2       0.93      0.93      0.93       540

    accuracy                           0.95      3900
   macro avg       0.95      0.94      0.94      3900
weighted avg       0.95      0.95      0.95      3900



### ***ProsusAI/finbert***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_paraphrased_imbalanced_ProsusAI_finbert_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_paraphrased_imbalanced_ProsusAI_finbert_finetuned.pth"
pretrained = 'ProsusAI/finbert'
run_cls(dataset, device, pretrained, model_file=model_file, train=False, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/488 [00:00<?, ?it/s]

Test loss is 0.168823
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      2260
           1       0.91      0.90      0.91      1100
           2       0.96      0.87      0.91       540

    accuracy                           0.94      3900
   macro avg       0.94      0.92      0.93      3900
weighted avg       0.94      0.94      0.94      3900



### ***dlxyuan/distilbert-base-multilingual-cased-sentiments-student***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pth"

pretrained = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***distilbert/distilbert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_paraphrased_imbalanced_distilbert-base-uncased_finetuned.pt'
pretrained = 'distilbert/distilbert-base-uncased'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_paraphrased_imbalanced_distilbert-base-uncased_finetuned.pth"
run_cls(dataset, device, pretrained, model_file=model_file, train=False, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Distillation...
Unfreezed the last layer successfully.


  0%|          | 0/488 [00:00<?, ?it/s]

Test loss is 0.565307
Classification Report
              precision    recall  f1-score   support

           0       0.79      0.89      0.84      2260
           1       0.70      0.67      0.68      1100
           2       0.79      0.41      0.54       540

    accuracy                           0.76      3900
   macro avg       0.76      0.66      0.69      3900
weighted avg       0.76      0.76      0.75      3900



### ***cross-encoder/nli-distilroberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_nli-distilroberta-base_finetuned.pt'
pretrained = 'cross-encoder/nli-distilroberta-base'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_nli-distilroberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pt'
pretrained = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/korean_imbalanced_FacebookAI_roberta-base_finetuned.pt'
pretrained = 'FacebookAI/roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_korean_imbalanced_roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=False, test_file_path=test_file_path)

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/488 [00:00<?, ?it/s]

Test loss is 0.263011
Classification Report
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      2260
           1       0.81      0.93      0.87      1100
           2       0.91      0.90      0.90       540

    accuracy                           0.91      3900
   macro avg       0.90      0.91      0.90      3900
weighted avg       0.92      0.91      0.91      3900



### ***FacebookAI/xlm-roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_FacebookAI_xlm-roberta-base_finetuned.pt'
pretrained = 'FacebookAI/xlm-roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_xlm-roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

# **French**

## **French Imbalanced OG Dataset (English)**

In [ ]:
filename = '/content/drive/MyDrive/LREC-Coling/Final_Data/French.csv'
data = pd.read_csv(filename)
data.drop_duplicates(inplace=True)
data.reset_index(inplace = True, drop=True)
data.head(5)

In [ ]:
data.rename(columns={'Text_english': 'feature', 'impact_level': 'class'}, inplace=True)
display(data['class'].value_counts())
dataset = data[['feature', 'class']]
key = {}
key['low'] = 0
key['medium'] = 1
key['high'] = 2
dataset['class'] = dataset['class'].apply(lambda x : key[x])
dataset

### ***bert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/french_imbalanced_bert_base_uncased_finetuned.pt'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_french_imbalanced_bert_base_uncased_finetuned.pth"

pretrained = 'bert-base-uncased'

run_cls(dataset, device, pretrained, model_file=model_file, train=False, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/82 [00:00<?, ?it/s]

Test loss is 0.903258
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.05      0.09       128
           1       0.51      1.00      0.68       326
           2       1.00      0.05      0.10       202

    accuracy                           0.52       656
   macro avg       0.84      0.37      0.29       656
weighted avg       0.76      0.52      0.39       656



### ***ProsusAI/finbert***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/french_imbalanced_ProsusAI_finbert_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_french_imbalanced_ProsusAI_finbert_finetuned.pth"

pretrained = 'ProsusAI/finbert'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***dlxyuan/distilbert-base-multilingual-cased-sentiments-student***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pth"

pretrained = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***distilbert/distilbert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-uncased_finetuned.pt'
pretrained = 'distilbert/distilbert-base-uncased'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-uncased_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***cross-encoder/nli-distilroberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_nli-distilroberta-base_finetuned.pt'
pretrained = 'cross-encoder/nli-distilroberta-base'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_nli-distilroberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pt'
pretrained = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/french_imbalanced_FacebookAI_roberta-base_finetuned.pt'
pretrained = 'FacebookAI/roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_french_imbalanced_roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/xlm-roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_FacebookAI_xlm-roberta-base_finetuned.pt'
pretrained = 'FacebookAI/xlm-roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_xlm-roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

## **French Imbalanced Paraphrased Dataset (English)**

In [ ]:
filename = '/content/drive/MyDrive/LREC-Coling/Paraphrased_Data/French_paraphrased.csv'
data = pd.read_csv(filename)
data.drop_duplicates(inplace=True)
data.reset_index(inplace = True, drop=True)
data.head(5)

,URL,news_title,news_content,impact_level,impact_length,news_title.1,Text_english,Text_japanese,Text_korean
0,https://www.novethic.fr/actualite/energie/tran...,Interdiction à la location des passoires therm...,"Depuis le 1er janvier, les passoires thermique...",low,More than 5 years,Prohibition for the rental of thermal colander...,Rental of the most energy-intensive thermal co...,1月1日以来、最もエネルギーを抑える熱colandersはもはやレンタルできません。目的は、...,1 월 1 일 이후 가장 에너지가 많은 열 콜란드를 더 이상 임대 할 수 없습니다....
1,https://www.novethic.fr/actualite/energie/tran...,Interdiction à la location des passoires therm...,"Depuis le 1er janvier, les passoires thermique...",low,More than 5 years,Prohibition for the rental of thermal colander...,Renting thermal colanders that consume too muc...,1月1日以来、最もエネルギーを抑える熱colandersはもはやレンタルできません。目的は、...,1 월 1 일 이후 가장 에너지가 많은 열 콜란드를 더 이상 임대 할 수 없습니다....
2,https://www.novethic.fr/actualite/energie/tran...,Interdiction à la location des passoires therm...,"Depuis le 1er janvier, les passoires thermique...",low,More than 5 years,Prohibition for the rental of thermal colander...,The rental of thermal colanders that require h...,1月1日以来、最もエネルギーを抑える熱colandersはもはやレンタルできません。目的は、...,1 월 1 일 이후 가장 에너지가 많은 열 콜란드를 더 이상 임대 할 수 없습니다....
3,https://www.novethic.fr/actualite/energie/tran...,Interdiction à la location des passoires therm...,"Depuis le 1er janvier, les passoires thermique...",low,More than 5 years,Prohibition for the rental of thermal colander...,"Since January 1, the most energy-intensive the...",1月1日以来、最もエネルギーを抑える熱colandersはもはやレンタルできません。目的は、...,1 월 1 일 이후 가장 에너지가 많은 열 콜란드를 더 이상 임대 할 수 없습니다....
4,https://www.novethic.fr/actualite/energie/tran...,Interdiction à la location des passoires therm...,"Depuis le 1er janvier, les passoires thermique...",low,More than 5 years,Prohibition for the rental of thermal colander...,Thermal colanders that require a lot of energy...,1月1日以来、最もエネルギーを抑える熱colandersはもはやレンタルできません。目的は、...,1 월 1 일 이후 가장 에너지가 많은 열 콜란드를 더 이상 임대 할 수 없습니다....


In [ ]:
data.rename(columns={'Text_english': 'feature', 'impact_level': 'class'}, inplace=True)
display(data['class'].value_counts())
dataset = data[['feature', 'class']]
key = {}
key['low'] = 0
key['medium'] = 1
key['high'] = 2
dataset['class'] = dataset['class'].apply(lambda x : key[x])
dataset

medium    1630
high      1010
low        640
Name: class, dtype: int64

<ipython-input-12-101060af5564>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['class'] = dataset['class'].apply(lambda x : key[x])


,feature,class
0,Rental of the most energy-intensive thermal co...,0
1,Renting thermal colanders that consume too muc...,0
2,The rental of thermal colanders that require h...,0
3,"Since January 1, the most energy-intensive the...",0
4,Thermal colanders that require a lot of energy...,0
...,...,...
3275,The increase in distances has not fully offset...,1
3276,"For a sustainable mobility policy, it is neces...",1
3277,A sustainable mobility policy necessitates red...,1
3278,"To achieve a sustainable mobility policy, it i...",1


### ***bert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/french_paraphrased_imbalanced_bert_base_uncased_finetuned.pt'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_french_paraphrased_imbalanced_bert_base_uncased_finetuned.pth"

pretrained = 'bert-base-uncased'

run_cls(dataset, device, pretrained, model_file=model_file, train=False, test_file_path=test_file_path)

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/410 [00:00<?, ?it/s]

Test loss is 0.205735
Classification Report
              precision    recall  f1-score   support

           0       0.94      0.91      0.93       640
           1       0.95      0.92      0.94      1630
           2       0.88      0.95      0.92      1010

    accuracy                           0.93      3280
   macro avg       0.93      0.93      0.93      3280
weighted avg       0.93      0.93      0.93      3280



### ***ProsusAI/finbert***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/french_imbalanced_ProsusAI_finbert_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_french_imbalanced_ProsusAI_finbert_finetuned.pth"

pretrained = 'ProsusAI/finbert'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Training starting...


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0 / 30: Train Loss: 1.161861, Train Accuracy: 0.2477, Val Loss: 1.154184, Val Accuracy: 0.2774
Epoch 0 / 30: Train Loss: 0.976165, Train Accuracy: 0.5290, Val Loss: 1.007273, Val Accuracy: 0.4878
Epoch 1 / 30: Train Loss: 0.906659, Train Accuracy: 0.5716, Val Loss: 0.971862, Val Accuracy: 0.5305
Epoch 2 / 30: Train Loss: 0.776285, Train Accuracy: 0.6338, Val Loss: 0.937092, Val Accuracy: 0.5610
Epoch 3 / 30: Train Loss: 0.556100, Train Accuracy: 0.7744, Val Loss: 0.744428, Val Accuracy: 0.6951
Epoch 4 / 30: Train Loss: 0.564135, Train Accuracy: 0.7599, Val Loss: 0.829138, Val Accuracy: 0.6311
Epoch 5 / 30: Train Loss: 0.436211, Train Accuracy: 0.8296, Val Loss: 0.722496, Val Accuracy: 0.7012
Epoch 6 / 30: Train Loss: 0.472813, Train Accuracy: 0.7969, Val Loss: 0.872128, Val Accuracy: 0.6372
Epoch 7 / 30: Train Loss: 0.284744, Train Accuracy: 0.9055, Val Loss: 0.660793, Val Accuracy: 0.7317
Epoch 8 / 30: Train Loss: 0.463038, Train Accuracy: 0.8155, Val Loss: 0.726885, Val Accurac

### ***dlxyuan/distilbert-base-multilingual-cased-sentiments-student***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pth"

pretrained = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***distilbert/distilbert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-uncased_finetuned.pt'
pretrained = 'distilbert/distilbert-base-uncased'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-uncased_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***cross-encoder/nli-distilroberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_nli-distilroberta-base_finetuned.pt'
pretrained = 'cross-encoder/nli-distilroberta-base'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_nli-distilroberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pt'
pretrained = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/french_paraphrased_imbalanced_FacebookAI_roberta-base_finetuned.pt'
pretrained = 'FacebookAI/roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_french_paraphrased_imbalanced_roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=True, test_file_path=test_file_path)

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


  0%|          | 0/410 [00:00<?, ?it/s]

Test loss is 0.249292
Classification Report
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       640
           1       0.96      0.88      0.92      1630
           2       0.87      0.95      0.91      1010

    accuracy                           0.91      3280
   macro avg       0.89      0.91      0.90      3280
weighted avg       0.91      0.91      0.91      3280



### ***FacebookAI/xlm-roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/french_paraphrased_imbalanced_FacebookAI_xlm-roberta-base_finetuned.pt'
pretrained = 'FacebookAI/xlm-roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_french_paraphrased_imbalanced_xlm-roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Unfreezing the last layer of BERT...
Unfreezed the last layer successfully.


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Training starting...


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0 / 30: Train Loss: 1.316653, Train Accuracy: 0.2077, Val Loss: 1.314256, Val Accuracy: 0.2256
Epoch 0 / 30: Train Loss: 1.284914, Train Accuracy: 0.2264, Val Loss: 1.281141, Val Accuracy: 0.2409
Epoch 1 / 30: Train Loss: 1.076123, Train Accuracy: 0.5008, Val Loss: 1.109612, Val Accuracy: 0.4939
Epoch 2 / 30: Train Loss: 0.963834, Train Accuracy: 0.5568, Val Loss: 1.004415, Val Accuracy: 0.4848
Epoch 3 / 30: Train Loss: 1.132700, Train Accuracy: 0.5008, Val Loss: 1.154929, Val Accuracy: 0.4939
Epoch 4 / 30: Train Loss: 1.048535, Train Accuracy: 0.5065, Val Loss: 1.094663, Val Accuracy: 0.4939
Epoch 5 / 30: Train Loss: 0.932802, Train Accuracy: 0.5404, Val Loss: 0.979382, Val Accuracy: 0.5061
Epoch 6 / 30: Train Loss: 0.772781, Train Accuracy: 0.6700, Val Loss: 0.894469, Val Accuracy: 0.5915
Epoch 7 / 30: Train Loss: 0.727199, Train Accuracy: 0.6978, Val Loss: 0.898336, Val Accuracy: 0.6098
Epoch 8 / 30: Train Loss: 0.706903, Train Accuracy: 0.7218, Val Loss: 0.883926, Val Accurac

# **Japanese**

## **Japanese Imbalanced OG Dataset (English)**

### ***bert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_bert_base_uncased_finetuned.pt'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_bert_base_uncased_finetuned.pth"

pretrained = 'bert-base-uncased'

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***ProsusAI/finbert***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_ProsusAI_finbert_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_ProsusAI_finbert_finetuned.pth"

pretrained = 'ProsusAI/finbert'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***dlxyuan/distilbert-base-multilingual-cased-sentiments-student***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pth"

pretrained = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***distilbert/distilbert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-uncased_finetuned.pt'
pretrained = 'distilbert/distilbert-base-uncased'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-uncased_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***cross-encoder/nli-distilroberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_nli-distilroberta-base_finetuned.pt'
pretrained = 'cross-encoder/nli-distilroberta-base'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_nli-distilroberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pt'
pretrained = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_FacebookAI_roberta-base_finetuned.pt'
pretrained = 'FacebookAI/roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/xlm-roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_FacebookAI_xlm-roberta-base_finetuned.pt'
pretrained = 'FacebookAI/xlm-roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_xlm-roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

## **Japanese Imbalanced Paraphrased Dataset (English)**

### ***bert-base-uncased***

In [ ]:
filename = '/content/drive/MyDrive/LREC-Coling/Paraphrased_Data/Japanese_paraphrased.csv'
data = pd.read_csv(filename)
data.drop_duplicates(inplace=True)
data.reset_index(inplace = True, drop=True)
data.head(5)

,ID,Text,Relevancy,ESG_type,Impact_type,Impact_duration,Text_english,Text_korean,Text_french
0,2376726,「環境レポーティング」、「事業における環境効率」など環境分野に関する取り組みについて高い評価...,Relevant,Environmental,Opportunity,within_2_years,"Our work on environmental issues, such as ""env...","우리는 ""환경보고""및 ""비즈니스 환경 효율성""과 같은 환경 분야에 대한 높은 찬사를...",Nous avons reçu des éloges sur les domaines en...
1,2376726,「環境レポーティング」、「事業における環境効率」など環境分野に関する取り組みについて高い評価...,Relevant,Environmental,Opportunity,within_2_years,We have been highly commended in regards to en...,"우리는 ""환경보고""및 ""비즈니스 환경 효율성""과 같은 환경 분야에 대한 높은 찬사를...",Nous avons reçu des éloges sur les domaines en...
2,2376726,「環境レポーティング」、「事業における環境効率」など環境分野に関する取り組みについて高い評価...,Relevant,Environmental,Opportunity,within_2_years,"Several areas of the environment, such as ""env...","우리는 ""환경보고""및 ""비즈니스 환경 효율성""과 같은 환경 분야에 대한 높은 찬사를...",Nous avons reçu des éloges sur les domaines en...
3,2376726,「環境レポーティング」、「事業における環境効率」など環境分野に関する取り組みについて高い評価...,Relevant,Environmental,Opportunity,within_2_years,Our efforts have been recognized in areas rela...,"우리는 ""환경보고""및 ""비즈니스 환경 효율성""과 같은 환경 분야에 대한 높은 찬사를...",Nous avons reçu des éloges sur les domaines en...
4,2376726,「環境レポーティング」、「事業における環境効率」など環境分野に関する取り組みについて高い評価...,Relevant,Environmental,Opportunity,within_2_years,"""Environmental reporting"" and environmental ef...","우리는 ""환경보고""및 ""비즈니스 환경 효율성""과 같은 환경 분야에 대한 높은 찬사를...",Nous avons reçu des éloges sur les domaines en...


In [ ]:
data.rename(columns={'Text_english': 'feature', 'Impact_type': 'class'}, inplace=True)
display(data['class'].value_counts())
dataset = data[['feature', 'class']]
key = {}
key['opportunity'] = 0
key['risk'] = 1
key['cannot distinguish'] = 2
dataset['class'] = dataset['class'].apply(lambda x : key[x])
dataset

Opportunity    235
Risk            15
Name: class, dtype: int64

KeyError: 'Opportunity'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/Japanese_paraphrased_imbalanced_bert_base_uncased_finetuned.pt'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_japanese_paraphrased_imbalanced_bert_base_uncased_finetuned.pth"

pretrained = 'bert-base-uncased'

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

NameError: name 'dataset' is not defined

### ***ProsusAI/finbert***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_ProsusAI_finbert_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_ProsusAI_finbert_finetuned.pth"

pretrained = 'ProsusAI/finbert'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***dlxyuan/distilbert-base-multilingual-cased-sentiments-student***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pt'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-multilingual-cased-sentiments-student_finetuned.pth"

pretrained = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***distilbert/distilbert-base-uncased***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilbert-base-uncased_finetuned.pt'
pretrained = 'distilbert/distilbert-base-uncased'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilbert-base-uncased_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***cross-encoder/nli-distilroberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_nli-distilroberta-base_finetuned.pt'
pretrained = 'cross-encoder/nli-distilroberta-base'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_nli-distilroberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pt'
pretrained = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_distilroberta-finetuned-financial-news-sentiment-analysis_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_FacebookAI_roberta-base_finetuned.pt'
pretrained = 'FacebookAI/roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)

### ***FacebookAI/xlm-roberta-base***

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_file = '/content/drive/MyDrive/LREC-Coling/Anubhav/models/english_imbalanced_FacebookAI_xlm-roberta-base_finetuned.pt'
pretrained = 'FacebookAI/xlm-roberta-base'

test_file_path = "/content/drive/MyDrive/LREC-Coling/Anubhav/models/test_dataset_english_imbalanced_xlm-roberta-base_finetuned.pth"

run_cls(dataset, device, pretrained, model_file=model_file, train=train, test_file_path=test_file_path)